# Census Housing by Age and State ETL

In this notebook data is extracted from the Census Bureau's American Community Survey 5-Year Data and loaded into a MongoDB database.

We will be using the Subject Tables, extracting variable from the DEMOGRAPHIC CHARACTERISTICS FOR OCCUPIED HOUSING UNITS group (S2502). The variables in question are percentages of Total Occupied Home, Owner Occupied Homes, and Renter Occupied Homes by age and state over the years 2010-2019.




In [1]:
#dependencies
import pandas as pd
import numpy as np
import pymongo
from census import Census
from config import census_key
api_key = census_key

### Extract and Transform

In [2]:
#transformation function to apply to every dataset 

def transform_census_age_data(census_data, year):
    
    data = census_data
    
    df = pd.DataFrame(data)
    
    rename_dict = {}

    for name in df.columns:
        code = name[-4:]
        if code == "011E":
            rename_dict[name] = "<35"
        elif code == "012E":
            rename_dict[name] = "35-44" 
        elif code == "013E":
            rename_dict[name] = "45-54"
        elif code == "014E":
            rename_dict[name] = "55-64"
        elif code == "015E":
            rename_dict[name] = "65-74"
        elif code == "016E":
            rename_dict[name] = "75-84"
        elif code == "017E":
            rename_dict[name] = ">85"

    df.rename(columns = rename_dict, inplace = True)
    
    df["year"] = year
    
    return df

In [3]:
# API calls and transformation

#dictionary to hold output
census_extract = {
    "totals" : [],
    "owner_occupied" : [],
    "renter_occupied" : []
}

for input_year in np.arange(2010,2020):
    
    c = Census(api_key, year=input_year)
    
    if input_year >= 2017: #accomodate for change in variable codes in 2017
        
        total_home_data = c.acs5st.get(("NAME", "S2502_C02_011E", "S2502_C02_012E", "S2502_C02_013E",
                              "S2502_C02_014E","S2502_C02_015E", "S2502_C02_016E", "S2502_C02_017E"), {'for': 'state:*'})

        owner_occupied_data = c.acs5st.get(("NAME", "S2502_C04_011E", "S2502_C04_012E", "S2502_C04_013E",
                              "S2502_C04_014E","S2502_C04_015E", "S2502_C04_016E", "S2502_C04_017E"), {'for': 'state:*'})
        
        renter_occupied_data = c.acs5st.get(("NAME", "S2502_C06_011E", "S2502_C06_012E", "S2502_C06_013E",
                              "S2502_C06_014E","S2502_C06_015E", "S2502_C06_016E", "S2502_C06_017E"), {'for': 'state:*'})
    else:
        
        total_home_data = c.acs5st.get(("NAME", "S2502_C01_011E", "S2502_C01_012E", "S2502_C01_013E",
                              "S2502_C01_014E","S2502_C01_015E", "S2502_C01_016E", "S2502_C01_017E"), {'for': 'state:*'})
        
        owner_occupied_data = c.acs5st.get(("NAME", "S2502_C02_011E", "S2502_C02_012E", "S2502_C02_013E",
                              "S2502_C02_014E","S2502_C02_015E", "S2502_C02_016E", "S2502_C02_017E"), {'for': 'state:*'})

        renter_occupied_data = c.acs5st.get(("NAME", "S2502_C03_011E", "S2502_C03_012E", "S2502_C03_013E",
                              "S2502_C03_014E","S2502_C03_015E", "S2502_C03_016E", "S2502_C03_017E"), {'for': 'state:*'})
    
    #apply transformation function
    total_home_df = transform_census_age_data(total_home_data, input_year)
    owner_occupied_df = transform_census_age_data(owner_occupied_data, input_year)
    renter_occupied_df = transform_census_age_data(renter_occupied_data, input_year)
    
    #append to output dictionary
    census_extract["totals"].append(total_home_df)
    census_extract["owner_occupied"].append(owner_occupied_df)
    census_extract["renter_occupied"].append(renter_occupied_df)


In [4]:
#combine arrays into one large daaframe for each set
census_combined = {key: pd.concat(value) for (key, value) in census_extract.items() }

census_combined["totals"]

,NAME,<35,35-44,45-54,55-64,65-74,75-84,>85,state,year
0,Alabama,20.1,18.4,21.0,18.0,12.3,7.7,2.5,01,2010
1,Alaska,23.9,20.0,25.1,18.6,8.1,3.5,0.8,02,2010
2,Arizona,22.5,18.8,19.7,17.0,12.0,7.5,2.6,04,2010
3,Arkansas,21.6,17.9,19.8,17.6,12.5,7.9,2.7,05,2010
4,California,20.7,21.1,22.2,16.8,9.9,6.6,2.6,06,2010
...,...,...,...,...,...,...,...,...,...,...
47,Washington,21.3,17.6,17.9,19.1,14.4,6.8,2.9,53,2019
48,West Virginia,16.0,14.9,17.5,20.8,17.6,9.4,3.7,54,2019
49,Wisconsin,19.9,16.1,18.2,20.1,14.5,7.7,3.6,55,2019
50,Wyoming,22.0,16.4,16.2,20.2,15.3,7.2,2.8,56,2019


In [37]:
df = census_combined["totals"]
df.groupby(["year"]).mean()

,<35,35-44,45-54,55-64,65-74,75-84,>85
year,,,,,,,
2010,20.980769,18.728846,21.428846,17.675000,11.086538,7.380769,2.736538
2011,20.721154,18.315385,21.273077,18.155769,11.355769,7.351923,2.821154
2012,20.415385,17.926923,21.071154,18.582692,11.759615,7.328846,2.907692
2013,20.171154,17.632692,20.761538,18.957692,12.192308,7.305769,2.994231
2014,19.988462,17.378846,20.376923,19.251923,12.644231,7.315385,3.046154
2015,19.826923,17.190385,19.946154,19.446154,13.138462,7.342308,3.101923
2016,19.663462,16.994231,19.534615,19.588462,13.680769,7.392308,3.151923
2017,19.630769,16.840385,19.069231,19.659615,14.157692,7.473077,3.165385
2018,19.519231,16.721154,18.605769,19.728846,14.611538,7.625000,3.200000


In [35]:
df = census_combined["renter_occupied"]
df.groupby(["year"]).mean()

,<35,35-44,45-54,55-64,65-74,75-84,>85
year,,,,,,,
2010,39.909615,19.175000,16.488462,10.786538,6.080769,4.784615,2.765385
2011,39.415385,19.076923,16.617308,11.186538,6.182692,4.694231,2.830769
2012,38.913462,19.057692,16.734615,11.575000,6.325000,4.532692,2.850000
2013,38.401923,19.109615,16.746154,11.930769,6.540385,4.425000,2.867308
2014,37.926923,19.144231,16.636538,12.269231,6.778846,4.369231,2.878846
2015,37.542308,19.194231,16.413462,12.580769,7.046154,4.332692,2.896154
2016,37.169231,19.128846,16.236538,12.855769,7.388462,4.321154,2.888462
2017,37.100000,18.976923,15.963462,13.034615,7.701923,4.363462,2.871154
2018,36.788462,18.900000,15.703846,13.242308,8.019231,4.451923,2.898077


In [36]:
df = census_combined["owner_occupied"]
df.groupby(["year"]).mean()

,<35,35-44,45-54,55-64,65-74,75-84,>85
year,,,,,,,
2010,12.098077,18.507692,23.713462,20.896154,13.430769,8.623077,2.742308
2011,11.780769,17.936538,23.473077,21.478846,13.825000,8.651923,2.846154
2012,11.346154,17.353846,23.176923,22.011538,14.421154,8.723077,2.982692
2013,11.026923,16.853846,22.761538,22.467308,15.025000,8.763462,3.092308
2014,10.771154,16.438462,22.286538,22.823077,15.659615,8.848077,3.176923
2015,10.559615,16.115385,21.778846,23.023077,16.332692,8.932692,3.242308
2016,10.398077,15.832692,21.271154,23.132692,17.023077,9.028846,3.321154
2017,10.480769,15.682692,20.692308,23.111538,17.546154,9.111538,3.351923
2018,10.484615,15.559615,20.136538,23.096154,18.046154,9.303846,3.378846


In [5]:
#convert to json format so it data can be loaded into MongoDB
census_as_dictionaries = []

for key in census_combined:
    new_index = census_combined[key].reset_index()
    dictionary = new_index.to_dict("records")
    census_as_dictionaries.append(dictionary)

### Load

In [6]:
#establish MongoDB connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db

#load in totals
db.total_housing_by_age.drop()
db.total_housing_by_age.find()

for record in census_as_dictionaries[0]:
    db.total_housing_by_age.insert_one(record)

#load in owner-occupied data
db.owner_occupied_housing_by_age.drop()
db.owner_occupied_housing_by_age.find()

for record in census_as_dictionaries[1]:
    db.owner_occupied_housing_by_age.insert_one(record)

    
#load in renter-occupied data
db.renter_occupied_housing_by_age.drop()
db.renter_occupied_housing_by_age.find()

for record in census_as_dictionaries[2]:
    db.renter_occupied_housing_by_age.insert_one(record)